 ## Flight Booking Agent

In [1]:
!pip uninstall boto3 botocore awscli --yes

Found existing installation: boto3 1.38.18
Uninstalling boto3-1.38.18:
  Successfully uninstalled boto3-1.38.18
Found existing installation: botocore 1.38.18
Uninstalling botocore-1.38.18:
  Successfully uninstalled botocore-1.38.18
Found existing installation: awscli 1.40.17
Uninstalling awscli-1.40.17:
  Successfully uninstalled awscli-1.40.17


In [2]:
# Install latest dependencies
!python -m pip install --force-reinstall --no-cache -q -r ../requirements.txt

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.2 requires nvidia-ml-py3==7.352.0, which is not installed.
dash 2.18.1 requires dash-core-components==2.0.0, which is not installed.
dash 2.18.1 requires dash-html-components==2.0.0, which is not installed.
dash 2.18.1 requires dash-table==5.0.0, which is not installed.
jupyter-ai 2.30.0 requires faiss-cpu!=1.8.0.post0,<2.0.0,>=1.8.0, which is not installed.
aiobotocore 2.21.1 requires botocore<1.37.2,>=1.37.0, but you have botocore 1.38.18 which is incompatible.
amazon-sagemaker-jupyter-ai-q-developer 1.1.0 requires numpy<=2.0.1, but you have numpy 2.2.6 which is incompatible.
amazon-sagemaker-sql-magic 0.1.4 requires numpy<2, but you have numpy 2.2.6 which is incompatible.
autogluon-common 1.2 requires numpy<2.1.4,>=1.25.0, but you have numpy 2.2.6 which is incompatible.
autogluon-core 1.2

In [3]:
!pip freeze | grep boto3

aioboto3 @ file:///home/conda/feedstock_root/build_artifacts/aioboto3_1742196379442/work
boto3==1.38.18


In [4]:
import uuid
import os
from pathlib import Path

def get_or_create_unique_resources_identifier():
    unique_resources_identifier_file = '../.u_resources_identifier'
    
    if os.path.exists(unique_resources_identifier_file):
        with open(unique_resources_identifier_file, 'r') as f:
            return f.read().strip()
    else:
        unique_resources_identifier = str(uuid.uuid4())[:8]
        with open(unique_resources_identifier_file, 'w') as f:
            f.write(unique_resources_identifier)
        return unique_resources_identifier

unique_resources_identifier = get_or_create_unique_resources_identifier()
resource_suffix = f"{unique_resources_identifier}"
print("Your resource suffix is", resource_suffix)

Your resource suffix is 348d2ff0


In [5]:
import boto3
import os
import json
import time
from datetime import datetime
from dateutil.relativedelta import relativedelta

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]

region = session.region_name

s3_client = boto3.client('s3', region)
bedrock_client = boto3.client('bedrock-runtime', region)

agent_foundation_model = [
    'anthropic.claude-3-5-sonnet-20240620-v1:0',
    'anthropic.claude-3-sonnet-20240229-v1:0',
    'anthropic.claude-3-haiku-20240307-v1:0',
    'amazon.titan-embed-text-v2:0',
    'amazon.titan-embed-image-v1',
    'amazon.titan-text-express-v1',
    'amazon.titan-text-lite-v1',
    'ai21.j2-mid-v1',
    'ai21.j2-ultra-v1',
    'cohere.command-text-v14',
    'cohere.embed-english-v3',
    'cohere.embed-multilingual-v3',
    'meta.llama2-13b-chat-v1',
    'meta.llama2-70b-chat-v1',
    'amazon.nova-pro-v1:0'
]

curr_month = datetime.now()

In [6]:
import sys

sys.path.insert(0, ".")
sys.path.insert(1, "..")

from utils.bedrock_agent_helper import (
    AgentsForAmazonBedrock
)
from utils.knowledge_base_helper import (
    KnowledgeBasesForAmazonBedrock
)
agents = AgentsForAmazonBedrock()
kb = KnowledgeBasesForAmazonBedrock()

In [7]:
# Define Flight Booking Agent resources
flight_agent_name = f"flight-agent-{resource_suffix}"
flight_lambda_name = f"fn-flight-agent-{resource_suffix}"
flight_agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{flight_agent_name}'

# Define DynamoDB tables for flights and bookings
flights_table = f"{flight_agent_name}-flights"
flights_pk = "flight_id"
flights_sk = "route"

bookings_table = f"{flight_agent_name}-bookings"
bookings_pk = "booking_id"
bookings_sk = "emp_id"

# Define arguments for DynamoDB tables
flights_table_args = [flights_table, flights_pk, flights_sk]
bookings_table_args = [bookings_table, bookings_pk, bookings_sk]


In [8]:
%%writefile flight_agent_lambda.py
import boto3
import json
import os
import uuid
from datetime import datetime, timedelta
from boto3.dynamodb.conditions import Key, Attr
from decimal import Decimal

# Initialize DynamoDB resources
dynamodb_resource = boto3.resource('dynamodb')
flights_table = os.getenv('flights_table')
flights_pk = os.getenv('flights_pk')
flights_sk = os.getenv('flights_sk')
bookings_table = os.getenv('bookings_table')
bookings_pk = os.getenv('bookings_pk')
bookings_sk = os.getenv('bookings_sk')

# Helper functions
def get_named_parameter(event, name):
    return next(item for item in event['parameters'] if item['name'] == name)['value']
    
def populate_function_response(event, response_body):
    return {'response': {'actionGroup': event['actionGroup'], 'function': event['function'],
                'functionResponse': {'responseBody': {'TEXT': {'body': str(response_body)}}}}}

def search_flights(origin, destination, departure_date, return_date=None):
    """Searches for available flights based on origin, destination and dates"""
    try:
        table = dynamodb_resource.Table(flights_table)
        
        # Create the route string (used as sort key)
        route = f"{origin}-{destination}"
        
        # Query for flights matching the route
        response = table.query(
            KeyConditionExpression=Key(flights_sk).eq(route)
        )
        
        flights = response.get('Items', [])
        
        # Filter flights by departure date
        matching_flights = []
        for flight in flights:
            flight_date = flight.get('departure_date')
            if flight_date == departure_date:
                matching_flights.append(flight)
        
        if not matching_flights:
            return {"status": "No flights found", "flights": []}
        
        # Sort flights by price (lowest first)
        matching_flights.sort(key=lambda x: float(x.get('price', '999999')))
        
        # Return top 5 cheapest flights
        top_flights = matching_flights[:5]
        
        return {
            "status": "Success",
            "flights": top_flights,
            "count": len(top_flights),
            "total_available": len(matching_flights)
        }
    except Exception as e:
        return {"status": "Error", "message": str(e)}

def check_eligibility(emp_id, flight_id):
    """Checks if an employee is eligible for a specific flight based on company policy"""
    try:
        # Get flight details
        flights_table_obj = dynamodb_resource.Table(flights_table)
        flight_response = flights_table_obj.get_item(
            Key={'flight_id': flight_id}
        )
        
        if 'Item' not in flight_response:
            return {"status": "Error", "message": "Flight not found"}
        
        flight = flight_response['Item']
        flight_class = flight.get('class', 'Economy')
        flight_price = float(flight.get('price', 0))
        
        # This would typically call the HR agent to check eligibility
        # For now, we'll implement basic rules:
        # - Economy class: All employees eligible
        # - Business class: Only Senior and Executive employees eligible
        # - First class: Only Executive employees eligible
        
        # In a real implementation, this would call the HR agent's API
        # For now, we'll simulate with basic rules
        
        # Get employee details (in a real implementation, this would come from HR agent)
        # Here we're simulating employee grades
        employee_grades = {
            "E001": "Senior",
            "E002": "Mid-level",
            "E003": "Junior",
            "E004": "Executive",
            "E005": "Executive"
        }
        
        employee_grade = employee_grades.get(emp_id, "Junior")
        
        eligible = True
        reason = "Eligible for booking"
        
        if flight_class == "Business" and employee_grade not in ["Senior", "Executive"]:
            eligible = False
            reason = "Only Senior and Executive employees are eligible for Business class"
        
        if flight_class == "First" and employee_grade != "Executive":
            eligible = False
            reason = "Only Executive employees are eligible for First class"
        
        # Price cap based on employee grade
        price_caps = {
            "Junior": 1000,
            "Mid-level": 2000,
            "Senior": 5000,
            "Executive": 10000
        }
        
        if flight_price > price_caps.get(employee_grade, 1000):
            eligible = False
            reason = f"Flight price exceeds the limit for {employee_grade} grade"
        
        return {
            "status": "Success",
            "eligible": eligible,
            "reason": reason,
            "flight": flight
        }
    except Exception as e:
        return {"status": "Error", "message": str(e)}

def book_flight(emp_id, flight_id):
    """Books a flight for an employee"""
    try:
        # First check eligibility
        eligibility = check_eligibility(emp_id, flight_id)
        
        if not eligibility.get("eligible", False):
            return {
                "status": "Error",
                "message": f"Not eligible for this flight: {eligibility.get('reason')}"
            }
        
        # Generate booking ID
        booking_id = str(uuid.uuid4())
        timestamp = datetime.now().isoformat()
        
        # Get flight details
        flight = eligibility.get("flight", {})
        
        # Create booking record
        booking = {
            "booking_id": booking_id,
            "emp_id": emp_id,
            "flight_id": flight_id,
            "status": "Confirmed",
            "created_at": timestamp,
            "origin": flight.get("origin"),
            "destination": flight.get("destination"),
            "departure_date": flight.get("departure_date"),
            "departure_time": flight.get("departure_time"),
            "arrival_time": flight.get("arrival_time"),
            "airline": flight.get("airline"),
            "flight_number": flight.get("flight_number"),
            "class": flight.get("class"),
            "price": flight.get("price")
        }
        
        # Save booking to DynamoDB
        table = dynamodb_resource.Table(bookings_table)
        table.put_item(Item=booking)
        
        return {
            "status": "Success",
            "booking_id": booking_id,
            "message": "Flight booked successfully",
            "booking_details": booking
        }
    except Exception as e:
        return {"status": "Error", "message": str(e)}

def generate_booking_document(booking_id):
    """Generates a booking confirmation document and stores it in S3"""
    try:
        # Get booking details
        table = dynamodb_resource.Table(bookings_table)
        response = table.get_item(
            Key={"booking_id": booking_id}
        )
        
        if 'Item' not in response:
            return {"status": "Error", "message": "Booking not found"}
        
        booking = response['Item']
        
        # In a real implementation, this would generate a PDF and upload to S3
        # For this example, we'll just return a simulated S3 URL
        
        document_url = f"https://s3.amazonaws.com/flight-bookings/{booking_id}.pdf"
        
        return {
            "status": "Success",
            "document_url": document_url,
            "booking_id": booking_id
        }
    except Exception as e:
        return {"status": "Error", "message": str(e)}

def lambda_handler(event, context):
    print(event)
    
    # Name of the function to invoke
    function = event.get('function', '')
    
    # Parameters to invoke function with
    parameters = event.get('parameters', [])
    
    # Route to appropriate function
    if function == 'search_flights':
        origin = get_named_parameter(event, "origin")
        destination = get_named_parameter(event, "destination")
        departure_date = get_named_parameter(event, "departure_date")
        
        # Return date is optional
        try:
            return_date = get_named_parameter(event, "return_date")
        except:
            return_date = None
            
        result = search_flights(origin, destination, departure_date, return_date)
    elif function == 'check_eligibility':
        emp_id = get_named_parameter(event, "emp_id")
        flight_id = get_named_parameter(event, "flight_id")
        result = check_eligibility(emp_id, flight_id)
    elif function == 'book_flight':
        emp_id = get_named_parameter(event, "emp_id")
        flight_id = get_named_parameter(event, "flight_id")
        result = book_flight(emp_id, flight_id)
    elif function == 'generate_booking_document':
        booking_id = get_named_parameter(event, "booking_id")
        result = generate_booking_document(booking_id)
    else:
        result = f"Error: Function '{function}' not recognized"

    # Format and return the response
    response = populate_function_response(event, result)
    print(response)
    return response


Writing flight_agent_lambda.py


In [9]:
flight_functions_def = [
    {
        "name": "search_flights",
        "description": """Searches for available flights based on origin, destination and dates""",
        "parameters": {
            "origin": {
                "description": "Origin airport code or city",
                "required": True,
                "type": "string"
            },
            "destination": {
                "description": "Destination airport code or city",
                "required": True,
                "type": "string"
            },
            "departure_date": {
                "description": "Departure date in YYYY-MM-DD format",
                "required": True,
                "type": "string"
            },
            "return_date": {
                "description": "Return date in YYYY-MM-DD format for round trips",
                "required": False,
                "type": "string"
            }
        }
    },
    {
        "name": "check_eligibility",
        "description": """Checks if an employee is eligible for a specific flight based on company policy""",
        "parameters": {
            "emp_id": {
                "description": "Employee ID",
                "required": True,
                "type": "string"
            },
            "flight_id": {
                "description": "Flight ID to check eligibility for",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        "name": "book_flight",
        "description": """Books a flight for an employee""",
        "parameters": {
            "emp_id": {
                "description": "Employee ID",
                "required": True,
                "type": "string"
            },
            "flight_id": {
                "description": "Flight ID to book",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        "name": "generate_booking_document",
        "description": """Generates a booking confirmation document and stores it in S3""",
        "parameters": {
            "booking_id": {
                "description": "Booking ID for which to generate document",
                "required": True,
                "type": "string"
            }
        }
    }
]


In [10]:
# Create sample flight data
flight_data = [
    {
        "flight_id": "FL001",
        "route": "NYC-LAX",
        "origin": "NYC",
        "destination": "LAX",
        "departure_date": "2024-08-15",
        "departure_time": "08:00",
        "arrival_time": "11:30",
        "airline": "American Airlines",
        "flight_number": "AA123",
        "class": "Economy",
        "price": "450.00",
        "seats_available": 25
    },
    {
        "flight_id": "FL002",
        "route": "NYC-LAX",
        "origin": "NYC",
        "destination": "LAX",
        "departure_date": "2024-08-15",
        "departure_time": "12:30",
        "arrival_time": "16:00",
        "airline": "Delta",
        "flight_number": "DL456",
        "class": "Economy",
        "price": "475.00",
        "seats_available": 18
    },
    {
        "flight_id": "FL003",
        "route": "NYC-LAX",
        "origin": "NYC",
        "destination": "LAX",
        "departure_date": "2024-08-15",
        "departure_time": "16:45",
        "arrival_time": "20:15",
        "airline": "United",
        "flight_number": "UA789",
        "class": "Economy",
        "price": "425.00",
        "seats_available": 12
    },
    {
        "flight_id": "FL004",
        "route": "NYC-LAX",
        "origin": "NYC",
        "destination": "LAX",
        "departure_date": "2024-08-15",
        "departure_time": "09:15",
        "arrival_time": "12:45",
        "airline": "JetBlue",
        "flight_number": "B6101",
        "class": "Business",
        "price": "1200.00",
        "seats_available": 8
    },
    {
        "flight_id": "FL005",
        "route": "NYC-LAX",
        "origin": "NYC",
        "destination": "LAX",
        "departure_date": "2024-08-15",
        "departure_time": "14:00",
        "arrival_time": "17:30",
        "airline": "American Airlines",
        "flight_number": "AA555",
        "class": "First",
        "price": "2500.00",
        "seats_available": 4
    },
    {
        "flight_id": "FL006",
        "route": "LAX-NYC",
        "origin": "LAX",
        "destination": "NYC",
        "departure_date": "2024-08-22",
        "departure_time": "07:30",
        "arrival_time": "15:45",
        "airline": "Delta",
        "flight_number": "DL222",
        "class": "Economy",
        "price": "465.00",
        "seats_available": 22
    },
    {
        "flight_id": "FL007",
        "route": "SFO-SEA",
        "origin": "SFO",
        "destination": "SEA",
        "departure_date": "2024-08-18",
        "departure_time": "10:00",
        "arrival_time": "12:15",
        "airline": "Alaska",
        "flight_number": "AS123",
        "class": "Economy",
        "price": "225.00",
        "seats_available": 30
    },
    {
        "flight_id": "FL008",
        "route": "SEA-SFO",
        "origin": "SEA",
        "destination": "SFO",
        "departure_date": "2024-08-25",
        "departure_time": "14:30",
        "arrival_time": "16:45",
        "airline": "Alaska",
        "flight_number": "AS456",
        "class": "Economy",
        "price": "235.00",
        "seats_available": 28
    }
]


In [11]:
# Create DynamoDB tables
agents.create_dynamodb(flights_table, flights_pk, flights_sk)
agents.create_dynamodb(bookings_table, bookings_pk, bookings_sk)

In [12]:
# Load sample flight data
agents.load_dynamodb(flights_table, flight_data)

In [13]:
# Create additional IAM policy for Lambda function
additional_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "dynamodb:GetItem",
                "dynamodb:PutItem",
                "dynamodb:DeleteItem",
                "dynamodb:Query",
                "dynamodb:Scan",
                "dynamodb:UpdateItem"
            ],
            "Resource": [
                f"arn:aws:dynamodb:{region}:{account_id}:table/{flights_table}",
                f"arn:aws:dynamodb:{region}:{account_id}:table/{bookings_table}"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "s3:PutObject",
                "s3:GetObject"
            ],
            "Resource": [
                f"arn:aws:s3:::flight-bookings/*"
            ]
        }
    ]
}


In [14]:
# Create the flight booking agent
flight_agent_instructions = """
You are the Flight Booking Agent. Your role is to assist users in booking flights for work travel. You will interact with the user to collect travel details, search for flights, apply eligibility rules, and finalize the booking.

Your capabilities include:
1. Searching for available flights based on origin, destination, and dates and show 5 lowest priced flight
2. Checking if employees are eligible for specific flights based on company policy
3. Booking flights for eligible employees
4. Generating booking confirmation documents

Core behaviors:
1. Always verify employee identity before providing personalized information
2. Maintain a professional yet conversational tone
3. Present flight options clearly with all relevant details
4. Enforce company travel policies consistently
5. Guide users through the booking process step by step
6. Provide clear explanations when a flight option is not eligible

Step-by-Step Workflow:
1. Collect Travel Inputs
   - Ask for origin, destination, and travel dates
   - Ask for any airline or time preferences
2. Search Flights
   - Use the searchFlights function with the provided inputs
   - Present the available options clearly
3. Apply Eligibility Rules
   - For each flight option, check eligibility based on employee grade and company policy
   - Filter out ineligible options and explain why
4. Present Options
   - Show top 3-5 eligible flights sorted by price
   - Include all relevant details (airline, times, class, price)
5. Book Flight
   - Once the user selects a flight, book it using the bookFlight function
   - Provide booking confirmation details
6. Generate Booking Document
   - Create a booking confirmation document
   - Provide the link to the document

Error Handling:
- If no flights are found, suggest alternative dates or routes
- If all flights are ineligible, explain the policy constraints
- If booking fails, provide clear error information and suggest next steps

Response style:
- Be helpful and solution-oriented
- Use clear, concise language
- Maintain natural conversation flow
- Provide all necessary flight details in an organized format
- Be thorough but avoid unnecessary details
"""

# Create the agent
flight_agent = agents.create_agent(
    agent_name=flight_agent_name,
    agent_description="Flight booking agent for corporate travel",
    agent_instructions=flight_agent_instructions,
    model_ids=agent_foundation_model,
    code_interpretation=False
)

# Add action group with Lambda function
agents.add_action_group_with_lambda(
    agent_name=flight_agent_name,
    lambda_function_name=flight_lambda_name,
    source_code_file="flight_agent_lambda.py",
    agent_functions=flight_functions_def,
    agent_action_group_name="flight_booking_actions",
    agent_action_group_description="Functions for searching, booking, and managing flights",
    additional_function_iam_policy=additional_policy,
    dynamo_args=flights_table_args
)


Table flight-agent-348d2ff0-flights already exists, skipping table creation step


In [15]:
# Create agent alias for multi-agent collaboration
flight_agent_alias_id, flight_agent_alias_arn = agents.create_agent_alias(
    flight_agent[0], 'v1'
)

In [16]:
# Store variables for use in other notebooks
flight_agent_arn = agents.get_agent_arn_by_name(flight_agent_name)
flight_agent_id = flight_agent[0]


In [17]:
%store flight_agent_arn
%store flight_agent_alias_arn
%store flight_agent_alias_id
%store flight_lambda_name
%store flight_agent_name
%store flight_agent_id

Stored 'flight_agent_arn' (str)
Stored 'flight_agent_alias_arn' (str)
Stored 'flight_agent_alias_id' (str)
Stored 'flight_lambda_name' (str)
Stored 'flight_agent_name' (str)
Stored 'flight_agent_id' (str)


## 6. Test the Flight Booking Agen
# Test the flight booking agent

In [19]:
%%time
response = agents.invoke(
    """I need to book a flight from NYC to LAX on August 15th. My employee ID is E001.""", 
    flight_agent_id, enable_trace=True
)
print("====================")
print(response)




invokeAgent API request ID: 2c3a5075-614c-4b46-88dd-2f8f79b7b920
invokeAgent API session ID: 4e9e148e-34bd-11f0-8aac-06efa4e78d78
---- Step 1 ----
Took 5.0s, using 1685 tokens (in: 1482, out: 203) to complete prior action, observe, orchestrate.
To book a flight for the user, I need to follow the step-by-step workflow. Let's start by collecting the necessary information and then search for available flights.

I have the following information:
- Origin: NYC
- Destination: LAX
- Departure Date: August 15th
- Employee ID: E001

Now, I'll use the flight_booking_actions__search_flights function to find available flights.
Using tool: search_flights with these inputs:
[{'name': 'origin', 'type': 'string', 'value': 'NYC'}, {'name': 'departure_date', 'type': 'string', 'value': '2023-08-15'}, {'name': 'destination', 'type': 'string', 'value': 'LAX'}]

--tool outputs:
{'status': 'Error', 'message': 'Required parameter name not set'}...

---- Step 2 ----
Took 4.6s, using 1871 tokens (in: 1712, out:

In [ ]:
# Test eligibility check
%%time
response = agents.invoke(
    """I'm employee E003 and I want to book flight FL004. Am I eligible?""", 
    flight_agent_id, enable_trace=True
)
print("====================")
print(response)